In [8]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets

import numpy as np
import pandas as pd

### Step1: Prepare datasets

In [7]:
(X_train, y_train), (X_test, y_test) = datasets.fashion_mnist.load_data()

In [11]:
class_names = ['tshirt', 'trousers', 'jumper', 'dress', 'coat',
               'sandals', 'shirt', 'trainers', 'bag', 'boots']

In [12]:
data = datasets.fashion_mnist.load_data()

In [138]:
dim1, dim2, dim3 = data[0][0].shape

In [120]:
data_df = pd.DataFrame(data[0][0].reshape((data[0][0].shape[0], 784)))

In [127]:
data_df = pd.concat([data_df, pd.Series(data[0][1], name='Y')], axis=1)

In [178]:
X_train = data_df[(data_df['Y'] != 0) & (data_df['Y'] != 5)].drop('Y', axis=1).values.reshape((-1, 28, 28))
y_train = data_df[(data_df['Y'] != 0) & (data_df['Y'] != 5)]['Y'].values

X_B = data_df[(data_df['Y'] == 0) | (data_df['Y'] == 5)].drop('Y', axis=1).values.reshape((-1, 28, 28))
y_B = data_df[(data_df['Y'] == 0) | (data_df['Y'] == 5)]['Y'].values
y_B = np.where(y_B == 5, 1, 0)

X_train_B = X_B[:200] 
y_train_B = y_B[:200]
X_test_B = X_B[200:]
y_test_B = y_B[200:]

In [151]:
# remove sandals and t-shirt

In [174]:
def eval_model(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train, epochs=10, verbose=0)
    
    print(model.evaluate(X_test, y_test, verbose=0))

### Step2: Train and test model on set B

In [182]:
def build_model_B():
    
    model = tf.keras.models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.BatchNormalization(),
        layers.Dense(units=50, activation='selu', kernel_initializer='lecun_normal'),
        layers.Dense(units=50, activation='selu', kernel_initializer='lecun_normal'),
        layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [191]:
model = build_model_B()

In [193]:
%%timeit
eval_model(model, X_train_B, y_train_B, X_test_B, y_test_B)

[0.15262560546398163, 0.9760169386863708]
[0.18044421076774597, 0.9761016964912415]
[0.19764047861099243, 0.9770339131355286]
[0.2177760899066925, 0.9761864542961121]
[0.23832030594348907, 0.9766949415206909]
[0.2993640899658203, 0.9744067788124084]
[0.3276752233505249, 0.9754237532615662]
[0.35265687108039856, 0.9754237532615662]
613 ms ± 5.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Step2: Fit and save model without t-hisrt and sandals

In [153]:
def build_model_A():
    
    model = tf.keras.models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.BatchNormalization(),
        layers.Dense(units=50, activation='selu', kernel_initializer='lecun_normal'),
        layers.Dense(units=50, activation='selu', kernel_initializer='lecun_normal'),
        layers.Dense(units=10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [155]:
model = build_model()

In [156]:
model.fit(X_train, y_train)

1500/1500 [==============================] - 3s 2ms/step - loss: 0.4154 - accuracy: 0.8509


In [157]:
model.save('fashion_mnist_model_A.h5')

In [163]:
for layer in range(len(model.get_weights())):
    print(model.get_weights()[layer].shape)

(784,)
(784,)
(784,)
(784,)
(784, 50)
(50,)
(50, 50)
(50,)
(50, 10)
(10,)


### Build Model_B based on Model_A

In [203]:
model_A = tf.keras.models.load_model('fashion_mnist_model_A.h5')

In [204]:
model_B_on_A = tf.keras.models.Sequential(model_A.layers[-1])
model_B_on_A.add(layers.Dense(1, activation='sigmoid'))

In [205]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable=False
    
model_B_on_A.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_B_on_A.fit(X_train_B, y_train_B, epochs=4, verbose=0)

In [209]:
for layer in model_B_on_A.layers:
    layer.trainable=True
    
model_B_on_A.compile(optimizer='adam', loss='binary_cross_entropy', metrics=['accuracy'])

model_B_on_A.fit(X_train_B, y_train_B, epochs=6, verbose=0)

(200, 28, 28)

In [ ]:
%%timeit
model_B_on_A.evaluate(X_test_B, y_test_B, verbose=10)